# Lecture 6 - Linear Systems 🧮
**Learning Objectives**
* Learn how to set up a system of linear equations as a matrix.
* Implement LU Decompositions to solve systems of linear equations.
* See how LU Decompositions allows you to efficiently solve systems of linear equations with changing loads.

## System of 5 Reactors with Unknown Concentrations (C&C Case Study 12.1)

<p align="center">
  <img src="https://github.com/cdefinnda/ECI-115_HW-Images/blob/main/cha32077_1203.png?raw=true" alt="cha32077_1203" width=500>
</p>

>* Reactor 1: $Q_{01}c_{01}+Q_{31}c_3=Q_{12}c_1+Q_{15}c_1 \rightarrow 6c_1-c_3=50$
* Reactor 2: $Q_{12}c_1=Q_{25}c_2+Q_{24}c_2+Q_{23}c_2 \rightarrow -3c_1+3c_2=0$
* Reactor 3: $-c_2+9c_3=160$
* Reactor 4: $-c_2-8c_3+11c_4-2c_5=0$
* Reactor 5: $-3c_1-c_2+4c_5=0$

### 💪 Set up system of equations and check if singular using determinant.

In [ ]:
import numpy as np
import scipy.linalg as sl

A = np.array([[], # Fill in components of Matrix A
              [],
              [],
              [],
              []])

b = np.array([]).T  # Fill in components of Array b

print('Shape of A:', A.shape) # Check dimensions of matrix
print('Shape of B:', b.shape) # Check dimensions of array
n = A.shape[0]                # Store # of rows of A for LU Decomposition

In [ ]:
# We inputed values into our system of equations as integers, but this can cause problems:
print('Float Division:', 3/2)     # Represents quotient as a float
print('Integer Division:', 3//2)  # Represents quotient as a integer (SIGNIFICANT roundoff error)

# To avoid this when we do matrix multiplication, convert all elements of A and b to float
A = A.astype(float)
b = b.astype(float)

In [ ]:
# Check if Matrix A is singular (i.e., det = 0)
print(sl.det(A))

### 💪 Perform LU decomposition without partial pivoting and compare to `scipy.linalg.lu`.

In [ ]:
# LU Decomposition
# Note: This does not include partial pivoting and may encounter divide by zero errors
# See C&C Figure 10.2 for pseudocode for LU with partial pivoting
L =   # Initialize L
U =   # Instialize U

for k in range(n-1):
    for i in range(k+1,n):
        L[i,k] = U[i,k] / U[k,k] # Multiplication Factor
        U[i,:] = U[i,:] - L[i,k] * U[k,:] # Modify row i based on pivot row k

❓ With this code, how would you handle a system of equations where there's a 0 in the diagonal of $\mathbf{U}$?
* [???]

In [ ]:
# Check Answers
# set precision for printing - easier to compare matrices
# suppress=True turns off scientific notation for small numbers
np.set_printoptions(precision=3, suppress=True)

In [ ]:
# Print Lower Diagonal
print(L)

In [ ]:
# Print Upper Diagonal
print(U)

In [ ]:
# Compare L @ U (Matrix Multiplication) to Matrix A
print(L @ U)
print('\n')
print(A)

In [ ]:
# Compare to scipy.linalg built in function lu()
# Note: The permute keyword makes the result comparable to ours
L_sl, U_sl = sl.lu(A, permute_l=True)
np.allclose(L, L_sl)
np.allclose(U, U_sl)

### 💪 Solve for unknowns and compare to `scipy.linalg.solve`.

Next we need to solve for the unknowns $\mathbf{c}$ given RHS vector $\mathbf{b}$. We can do this in two steps:
- Solve $\mathbf{Ld} = \mathbf{b}$ for the intermediate vector $\mathbf{d}$ using forward substitution
- Solve $\mathbf{Ux} = \mathbf{d}$ using back substitution

In [ ]:
# Inspect System for Conducting Ld = b
print(L)
print('\n')
print('b = ',b) # This array is actually vertical.

In [ ]:
# Forward Substitution
d = np.zeros(n)
d[0] = b[0] / L[0,0]

for i in range(1,n):
    d[i] = # Implement the iterative forward substitution operation here.

In [ ]:
# Inspect System for Conducting Ux = d
print(U)
print('\n')
print('d = ',d) # This array is actually vertical.

In [ ]:
# Back Substitution
x = np.zeros(n)
x[-1] = d[-1] / U[-1,-1] # index -1 for last element

for i in range(n-2,-1,-1): # loop backward starting from second-to-last row
    x[i] = # Implement the iterative back substitution operation here.

In [ ]:
# Compare Solution to sl.solve(A,b)
print('Our answer: ', x)
print('Matches Scipy? ', np.allclose(x, sl.solve(A,b)))

### 💪 Change b (new concentration inputs) and solve again without repeating the row elimination.

Now if the input concentration $\mathbf{b}$ changes, Gauss elimination would require the row elimination to be repeated with $O(n^3)$ operations. However, since we already have the LU decomposition, we only need to perform the forward/back substitution steps. This is $O(n^2)$ operations.

For example, say concentration $c_{01}=20$ and $c_{03}=10$. Using mass balance, this would lead to a new vector $\mathbf{b} = [100, 0, 80, 0, 0]^T$. We are only changing the inputs, so the matrix $\mathbf{A}$ does not change.

❓ How do you expect this change to impact the steady concentrations of your system?
* [???]

In [ ]:
b2 = np.array([]).T # Fill in new values for b here.

# Repeat the steps above to solve (could write this in a function)

# Forward Substitution
d2 = np.zeros(n)
d2[0] = b2[0] / L[0,0]

for i in range(1,n):
    d2[i] = (b2[i] - L[i,:] @ d2) / L[i,i]

# Back Substitution
x2 = np.zeros(n)
x2[-1] = d2[-1] / U[-1,-1] # index -1 for last element

for i in range(n-2,-1,-1): # loop backward starting from second-to-last row
    x2[i] = (d2[i] - U[i,i+1:n] @ x2[i+1:n]) / U[i,i]

print('Our answer: ', x2)
print('Matches Scipy? ', np.allclose(x2, sl.solve(A,b2)))

In [ ]:
# Compare Results from Two Different Concentration Loadings
print(x, '(Original Steady State Concentrations)')
print(x2, '(New Steady State Concentrations)')

❓ How did this change to impact the steady concentrations of your system?
* [???]

In the next lecture we will work on more formal ways to analyze the change in system states as a linear function of the forcing using the matrix inverse.